In [ ]:
%load_ext autoreload
%autoreload
from tune_and_draw import TuneAndDraw
from fedhybrid_innerloops import FedHybrid
from parameter import TuneParam,TrainParam
from costFunc import LogisticCostFunc
import numpy as np
import random
from sklearn.datasets import load_boston
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd
import numpy as np
import random
from scipy.optimize import minimize

In [ ]:
def dataset(nclient): 
    np.random.seed(2022)
    random.seed(2022)
    df =  pd.read_table('agaricus-lepiota.data', delimiter=',', header=None)
    column_labels = [
            'class', 'cap shape', 'cap surface', 'cap color', 'bruised', 'odor',
            'gill attachment', 'gill spacing', 'gill size', 'gill color', 
            'stalk shape', 'stalk root', 'stalk surface above ring',
            'stalk surface below ring', 'stalk color above ring',
            'stalk color below ring', 'veil type', 'veil color', 'ring number',
            'ring type', 'spore print color', 'population', 'habitat']

    df.columns = column_labels
    
        # excluding any training example that has missing values for stalk root.
    df = df[df['stalk root'] != '?']

    X = df.loc[:, df.columns != 'class']
    y = df['class'].to_frame()

    # Encoding categorical features
    X_enc = pd.get_dummies(X)
    # Standardizing the features
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X_enc)

    # Encoding the target variable
    le = LabelEncoder()
    y_enc = le.fit_transform(y.values.ravel())

    X_train, X_test, y_train, y_test = train_test_split(X_std, y_enc, test_size=0.3, stratify=y_enc, random_state=42)

    # number of examples
    #n=354# same as quadratic housing
    n = X_train.shape[0]
    # ndim
    ndim = X_train.shape[1] + 1
    # number of classes
    k = 2
    label = [0,1]

    intercept = np.ones((X_train.shape[0], 1))
    X_train = np.hstack((intercept, X_train))
    intercept = np.ones((X_test.shape[0], 1))
    X_test = np.hstack((intercept, X_test))

    # Not uniformly distributed among clients

    X_train, y_train = shuffle(X_train, y_train)    

    X_df = pd.DataFrame([y_train, X_train])

    X_df = X_df.T
    X_df.columns = ['y_train', 'X_train']

    X_df = X_df.reset_index()
    #print(X_df.columns)
    #X_df=X_df.iloc[:354]

    # Split it into different groups according to the label
    X_df_0 = X_df[X_df['y_train']==0]
    X_df_1 = X_df[X_df['y_train']==1]
    #print('len(X_df_0),len(X_df_1)',len(X_df_0),len(X_df_1))
    #ni=[45, 38, 29, 12, 62, 76, 36, 56]#same as quadratic housing
    p = np.zeros(nclient)
    s = 0
    for i in range(nclient):
        p[i] = np.random.lognormal(4, 2) + 50
        s += p[i]
    p = p/s
    ni = p * n
    ni = [int(num) for num in ni]
    ni[-1] = ni[-1] + n - sum(ni) 
    X_df_reorder = pd.concat([X_df_0, X_df_1])
    indices_set = [[] for i in range(nclient)]
    s = 0
    for j in range(nclient):
        indices_set[j] =  X_df_reorder['index'].to_list()[s:s+ni[j]]
        s += ni[j]
    A = [[] for _ in range(nclient)]
    y = [[] for _ in range(nclient)]
    for i in range(nclient):
        A[i] = X_train[indices_set[i]]
        y[i] = y_train[indices_set[i]]

    for i in range(nclient):
        print(ni[i])
    return A,y

In [ ]:
def prepare(nclient):
    A,y =dataset(nclient)
    gamma = 1
    func = LogisticCostFunc(A, y, gamma)

    initial_x =np.zeros([func.ndim,1])
    fn_min = minimize(func.global_func, initial_x, tol=1e-30) #result object
    fn_star = fn_min.fun
    method = FedHybrid(func, fn_star)
    tuneAndDraw = TuneAndDraw(nclient, initial_x,setup=4)
    return method,tuneAndDraw

synchronous logistic mushroom

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 1
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/4 synchronous logistic mushroom.pdf"
tuneAndDraw.draw(df,method.train_innerloop_lambdaBefore,"synchronous logistic mushroom",filename,save = 1,asyn=0,possionBeta=[])

innerloop 4 logistic mushroom

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 4
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/4 innerloop 4 logistic mushroom 2.pdf"
tuneAndDraw.draw(df,method.train_innerloop_lambdaBefore,"logistic mushroom innerloop 4",filename,save = 0,asyn=0,possionBeta=[])

async logistic mushroom

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 0
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/4 asynchronous logistic mushroom.pdf"
tuneAndDraw.draw(df,method.asynchronousTrain2,"asynchronous logistic mushroom",filename,save = 1,asyn=1,possionBeta=[5, 10, 4, 1, 2, 7, 100, 5])